# Graph Embedding

주요 목적
 - 1. 노드 간의 관계 유지 : 그래프 구조를 반영하여 유사한 노드들이 벡터 공간에서도 가까운 위치를 갖도록 함.
 - 2. 이웃 정보의 반영 : 노드가 속한 지역적 구조(이웃 관계)를 벡터 표현에 포함시켜, 그래프의 중요한 패턴을 보존
 - 3. 전역 구조 표현 : 노드뿐만 아니라 전체 그래프의 패턴을 학습하여, 그래프 수준의 분석이 가능하도록 함.

### 여기에 Graph Embedding 내용 + DeepWalk, Node2Vec 내용 추가하기

### Graph Embedding practice

In [1]:
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from gensim.models import Word2Vec
from sklearn.manifold import TSNE

In [4]:
# Create Sample Graph

def create_sample_graph():
    G = nx.karate_club_graph()
    return G

# Correctness 확인
G = create_sample_graph()
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")
assert G.number_of_nodes() == 34, "Karate club graph should have 34 nodes"
# assert는 조건이 참인지 확인하는 구문, assert 조건식, "에러 메시지"
# if 조건식이 False 이면, 에러 메시지가 출력됨

Number of nodes: 34
Number of edges: 78


In [6]:
# DeepWalk Implementation

def random_walk(G, start_node, length):
    walk = [start_node]
    for _ in range(length):
        curr = walk[-1]
        neighbors = list(G.neighbors(curr))
        if neighbors:
            walk.append(random.choice(neighbors))
    return [str(node) for node in walk]

def deep_walk(G, walk_length, num_walks, dimensions):
    walks = []
    nodes = list(G.nodes())
    
    for _ in range(num_walks):
        random.shuffle(nodes)
        for node in nodes:
            walk = random_walk(G, node, walk_length)
            walks.append(walk)
            
        model = Word2Vec(walks, vector_size=dimensions, window=5, min_count=0, sg=1, workers=4)
        return model
    
# Correctness 확인
test_walk = random_walk(G, 0, 5)
print(f"Sample random walk: {test_walk}")

Sample random walk: ['0', '5', '6', '0', '7', '0']


In [ ]:
# Node2Vec implementation

def biased_random_walk(G, start_node, length, p, q):
    walk = [start_node]
    for _ in range(length):
        curr = walk[-1]
        prev = walk[-2] if len(walk) > 1 else None
        neighbors = list(G.neighbors(curr))
        if not neighbors:
            break
        
        if prev is None:
            walk.append(random.choice(neighbors))
        else:
            next_node = sample_next_node(G, curr, prev, neighbors, p, q)
            walk.append(next_node)
    return [str(node) for node in walk]

def sample_next_node(G, curr, prev, neighbors, p, q):
    return random.choice(neighbors)

def node2vec(G, walk_length, num_walks, dimensions, p, q):
    walks = []
    nodes = list(G.nodes())
    
    for _ in range(num_walks):
        random.shuffle(nodes)
        for node in nodes:
            walk = biased_random_walk(G, node, walk_length, p, q)
            walks.append(walk)
            
    model = Word2Vec(walks, vector_size=dimensions, window=5, min_count=0, sg=1, workers=4)
    return model

# Correctness 확인
test_biased_walk = biased_random_walk(G, 0, 5, 1, 1)
print(f"Sample biased random walk: {test_biased_walk}")

Sample biased random walk: ['0', '7', '1', '30', '1', '7']
